<a href="https://colab.research.google.com/github/jimin0/Graduation_Project_HE_Study/blob/main/ex3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone https://github.com/Lab41/PySEAL.git
%cd SEAL-Python

# Install dependencies
!pip3 install numpy pybind11

# Init the SEAL and pybind11
!git submodule update --init --recursive
# Get the newest repositories (dev only)
# git submodule update --remote

# Build the SEAL lib
%cd SEAL
!cmake -S . -B build -DSEAL_USE_MSGSL=OFF -DSEAL_USE_ZLIB=OFF
!cmake --build build
%cd ..

# Run the setup.py
!python3 setup.py build_ext -i


Cloning into 'PySEAL'...
remote: Enumerating objects: 1348, done.
remote: Total 1348 (delta 0), reused 0 (delta 0), pack-reused 1348
Receiving objects: 100% (1348/1348), 1.32 MiB | 3.76 MiB/s, done.
Resolving deltas: 100% (906/906), done.
/content/PySEAL
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.7/227.7 kB 2.7 MB/s eta 0:00:00
/content/PySEAL/SEAL
CMake Error: The source directory "/content/PySEAL/SEAL" does not appear to contain CMakeLists.txt.
Specify --help for usage, or press the help button on the CMake GUI.
Error: /content/PySEAL/SEAL/build is not a directory
/content/PySEAL
python3: can't open file '/content/PySEAL/setup.py': [Errno 2] No such file or directory


일단 seal-python으로

In [ ]:
!git clone https://github.com/Huelse/SEAL-Python.git
%cd SEAL-Python

# Install dependencies
!pip3 install numpy pybind11

# Init the SEAL and pybind11
!git submodule update --init --recursive
# Get the newest repositories (dev only)
# git submodule update --remote

# Build the SEAL lib
%cd SEAL
!cmake -S . -B build -DSEAL_USE_MSGSL=OFF -DSEAL_USE_ZLIB=OFF
!cmake --build build
%cd ..

# Run the setup.py
!python3 setup.py build_ext -i


Cloning into 'SEAL-Python'...
remote: Enumerating objects: 1628, done.
remote: Counting objects: 100% (252/252), done.
remote: Compressing objects: 100% (86/86), done.
remote: Total 1628 (delta 176), reused 191 (delta 162), pack-reused 1376
Receiving objects: 100% (1628/1628), 8.68 MiB | 9.14 MiB/s, done.
Resolving deltas: 100% (882/882), done.
/content/PySEAL/SEAL-Python
Submodule 'SEAL' (https://github.com/microsoft/SEAL.git) registered for path 'SEAL'
Submodule 'pybind11' (https://github.com/pybind/pybind11.git) registered for path 'pybind11'
Cloning into '/content/PySEAL/SEAL-Python/SEAL'...
Cloning into '/content/PySEAL/SEAL-Python/pybind11'...
Submodule path 'SEAL': checked out '206648d0e4634e5c61dcf9370676630268290b59'
Submodule path 'pybind11': checked out 'be97c5a98b4b252c524566f508b5c79410d118c6'
/content/PySEAL/SEAL-Python/SEAL
-- Build type (CMAKE_BUILD_TYPE): Release
-- The CXX compiler identification is GNU 11.4.0
-- The C compiler identification is GNU 11.4.0
-- Detectin

In [ ]:
# SEAL 라이브러리의 모든 클래스와 함수를 임포트합니다.
from seal import *
import numpy as np

# BGV 암호화 스킴을 사용할 암호화 매개변수 객체를 생성합니다.
parms = EncryptionParameters(scheme_type.bgv)

# 다항식 모듈러스의 차수를 설정합니다.
# 이는 암호화된 다항식의 최대 차수를 정의하고,
# 암호화의 보안 수준과 성능에 영향을 미칩니다.
"""
poly_modulus_degree가 클수록 암호화된 텍스트의 크기가 커지고 모든 연산이 느려지지만,
더 복잡한 암호화된 계산을 가능하게 함. 권장되는 값들은 1024, 2048, 4096, 8192, 16384, 32768이지만,
이 범위를 넘어가는 것도 가능
"""
poly_modulus_degree = 8192
parms.set_poly_modulus_degree(poly_modulus_degree)
parms.set_coeff_modulus(CoeffModulus.BFVDefault(poly_modulus_degree))
parms.set_plain_modulus(PlainModulus.Batching(poly_modulus_degree, 20))


# 설정된 암호화 매개변수를 기반으로 SEAL 컨텍스트를 생성합니다.
# 이 컨텍스트는 암호화, 복호화, 키 생성 등에 사용됩니다.
context = SEALContext(parms)


# 계수 모듈러스를 설정합니다. 이는 암호화 시 사용되는 모듈러스 값으로,
# BFVDefault는 보안 수준에 맞는 기본 모듈러스 값을 선택합니다.
parms.set_coeff_modulus(CoeffModulus.BFVDefault(poly_modulus_degree))

# 평문 모듈러스를 설정합니다.
# 이 모듈러스는 암호화되지 않은 데이터가 취할 수 있는 값의 범위를 정의합니다.
# 이 경우, 값의 범위는 0에서 255입니다.
parms.set_plain_modulus(256)


* 		bootstrapping X: 첫번째 conv layer 까지만 동형암호 연산으로 하고 --> Relu, maxpooling은 사용자가 처리 (복호화한 후) --> maxpooling 결과를 암호화하지 않고 서버로 보내서 그 이후 layer 처리는 plain text로 연산
* 		TODO: conv layer 연산을 bootstrapping 없이 다시 복호가 잘 되는지 확인 (pySEAL)
* 		모델: 첫번째 conv layer만 동형암호 연산 나머지는 original neural network로 처리

In [ ]:
# 키 생성

keygen = KeyGenerator(context)

secret_key = keygen.secret_key()
public_key = keygen.create_public_key()
relin_keys = keygen.create_relin_keys()
galois_keys =  keygen.create_galois_keys()

encryptor = Encryptor(context, public_key)
evaluator = Evaluator(context)
decryptor = Decryptor(context, secret_key)


batch_encoder = BatchEncoder(context)
slot_count = batch_encoder.slot_count()
row_size = slot_count / 2
print(f'Plaintext matrix row size: {row_size}')

Plaintext matrix row size: 4096.0


In [ ]:
import numpy as np

class ConvLayer:
    def __init__(self, input_shape, num_filters, kernel_size, padding, stride):
        self.input_height, self.input_width, self.num_channels = input_shape
        self.num_filters = num_filters
        self.kernel_size = kernel_size
        self.padding = padding
        self.stride = stride

        # Initialize filters and bias with random values
        self.filters = np.random.randn(num_filters, kernel_size, kernel_size, self.num_channels)
        self.bias = np.random.randn(num_filters)

    def conv_forward(self, input_volume):
        input_volume_padded = np.pad(
            input_volume,
            ((self.padding, self.padding), (self.padding, self.padding), (0, 0)),
            mode='constant'
        )

        output_height = ((self.input_height - self.kernel_size + 2 * self.padding) // self.stride) + 1
        output_width = ((self.input_width - self.kernel_size + 2 * self.padding) // self.stride) + 1

        output_volume = np.zeros((output_height, output_width, self.num_filters))

        for h in range(output_height):
            for w in range(output_width):
                for f in range(self.num_filters):
                    h_start = h * self.stride
                    h_end = h_start + self.kernel_size
                    w_start = w * self.stride
                    w_end = w_start + self.kernel_size

                    receptive_field = input_volume_padded[h_start:h_end, w_start:w_end, :]
                    output_volume[h, w, f] = np.sum(receptive_field * self.filters[f]) + self.bias[f]


        print("Output Height:", output_height)
        print("Output Width:", output_width)
        print("output_volume : ", output_volume)
        return output_volume




Input Shape : (7, 7, 2) Output Shape : (7, 7, 1) K : (3, 3) P : (1, 1) S : (1, 1) Filter : 1



https://velog.io/@groovallstar/cnn

In [ ]:
# Example usage
input_shape = (7, 7, 2)
num_filters = 1
kernel_size = 3
padding = 1
stride = 1

conv_layer = ConvLayer(input_shape, num_filters, kernel_size, padding, stride)

# Example input
input_volume = np.random.randn(7, 7, 2)
output_volume = conv_layer.conv_forward(input_volume)

print("Output Volume Shape:", output_volume.shape)

Output Height: 7
Output Width: 7
output_volume :  [[[ 0.91972163]
  [ 4.25710822]
  [ 5.14886884]
  [ 2.83060296]
  [ 2.10231693]
  [-1.30577925]
  [-0.5528013 ]]

 [[-0.39192367]
  [ 2.36252896]
  [ 1.31615745]
  [ 9.79761403]
  [ 7.33992003]
  [-3.4670083 ]
  [-2.12416378]]

 [[ 1.24572377]
  [ 2.62349224]
  [-5.32700824]
  [-8.2891614 ]
  [11.04895178]
  [-0.01553576]
  [ 0.52901699]]

 [[ 7.35272762]
  [11.83450613]
  [ 5.27409802]
  [-1.42655864]
  [ 2.31752122]
  [11.48337479]
  [-2.83449447]]

 [[ 0.48401721]
  [ 3.13575233]
  [12.54590157]
  [-0.79747014]
  [ 2.42357428]
  [ 1.5477765 ]
  [ 4.59849384]]

 [[-6.23768295]
  [ 3.48892756]
  [ 4.71320477]
  [-2.36082164]
  [ 3.6910405 ]
  [ 2.69424522]
  [ 4.933481  ]]

 [[ 1.11014392]
  [-0.25046223]
  [ 2.74556767]
  [ 1.43955266]
  [-3.72349602]
  [ 1.46145342]
  [ 5.73907777]]]
Output Volume Shape: (7, 7, 1)


# After

output_height = ((input_height - kernel_size + 2 * padding) // stride) + 1

output_width = ((input_width - kernel_size + 2 * padding) // stride) + 1


-> 암호화

In [ ]:
import numpy as np

class ConvLayer:
    def __init__(self, input_shape, num_filters, kernel_size, padding, stride):
        self.input_height, self.input_width, self.num_channels = input_shape
        self.num_filters = num_filters
        self.kernel_size = kernel_size
        self.padding = padding
        self.stride = stride

        # Initialize filters and bias with random values
        self.filters = np.random.randn(num_filters, kernel_size, kernel_size, self.num_channels)
        self.bias = np.random.randn(num_filters)

    def conv_forward(self, input_volume):
        input_volume_padded = np.pad(
            input_volume,
            ((self.padding, self.padding), (self.padding, self.padding), (0, 0)),
            mode='constant'
        )

        # output_height = ((self.input_height - self.kernel_size + 2 * self.padding) // self.stride) + 1
        # output_width = ((self.input_width - self.kernel_size + 2 * self.padding) // self.stride) + 1


        output_volume = np.zeros((output_height, output_width, self.num_filters))

        for h in range(output_height):
            for w in range(output_width):
                for f in range(self.num_filters):
                    h_start = h * self.stride
                    h_end = h_start + self.kernel_size
                    w_start = w * self.stride
                    w_end = w_start + self.kernel_size

                    receptive_field = input_volume_padded[h_start:h_end, w_start:w_end, :]
                    output_volume[h, w, f] = np.sum(receptive_field * self.filters[f]) + self.bias[f]


        print("Output Height:", output_height)
        print("Output Width:", output_width)
        print("output_volume : ", output_volume)
        return output_volume




In [ ]:
def homomorphic_output_height(context, input_height, kernel_size, padding, stride):
    # output_height = ((self.input_height - self.kernel_size + 2 * self.padding) // self.stride) + 1    --> 암호화?


    # Plaintext
    input_height_plain = Plaintext(input_height)
    kernel_size_plain = Plaintext(kernel_size)
    padding_plain = Plaintext(padding)
    stride_plain = Plaintext(stride)

    # 결과 저장할 chiphertext 객체
    output_height_encrypted = Ciphertext()

    # chiphertext 생성 : input_height, kernel_size, padding, stride
    input_height_encrypted = evaluator.encrypt(input_height_plain)
    kernel_size_encrypted = evaluator.encrypt(kernel_size_plain)
    padding_encrypted = evaluator.encrypt(padding_plain)
    stride_encrypted = evaluator.encrypt(stride_plain)

    # Calculate numerator
    num_encrypted = encryptor.encrypt(Plaintext("2"))

    # 2 * self.padding
    numerator = Ciphertext()
    numerator = evaluator.multiply(num_encrypted, num_encrypted)
    evaluator.relinearize_inplace(numerator, relin_keys)


    # self.kernel_size + (2 * self.padding)
    evaluator.add_inplace(numerator, kernel_size_encrypted, numerator)

    numerator = evaluator.multiply_plain(input_height_encrypted, stride_encrypted)
    evaluator.add_plain_inplace(numerator, -kernel_size_encrypted)
    evaluator.add_plain_inplace(numerator, padding_encrypted, padding_encrypted)

    # Calculate denominator
    denominator = evaluator.encrypt(stride)

    # Calculate output_height
    output_height_encrypted = evaluator.divide(numerator, denominator)

    # Convert output_height ciphertext to plaintext
    output_height_plain = Plaintext()
    evaluator.decrypt(output_height_encrypted, output_height_plain)

    # Decode output_height from plaintext
    output_height = context.decode(output_height_plain)

    return output_height
